In [1]:
#!pip install yfinance
#!pip install pandas
#!pip install chardet
#!pip install pandas_datareader
#!pip install YahooFinancials
#!pip install scikit-learn
#!pip install --upgrade yfinance
#!pip install tejapi
#!pip install xlsxwirter
import os
import yfinance as yf
import pandas as pd
import numpy as np
import requests
import pandas_datareader as pdr
import chardet 
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

# 年份處理與資料初整理

In [2]:
# 指定檔案路徑
file_path = r"E:\碩士\財務工程(二)\FZ分數2.xlsx"

# 讀取 Excel 檔案
basic_data = pd.read_excel(file_path)

basic_data["現金增資(仟股)"] = basic_data["現金增資(仟股)"].fillna(0)

basic_data["年月"] = basic_data["年月"].astype(str).str.slice(0, 4)

basic_data["股價"] = basic_data["當季季底P/B"] * basic_data["每股淨值(A)"]

# 假設 basic_data 已經讀取進來
basic_data['報酬率'] = basic_data.groupby('公司')['股價'].pct_change(fill_method=None).ffill()

# 移除 "收盤價(元)" 欄位
basic_data = basic_data.drop(columns=['股價'])

display(basic_data)

,公司,年月,流動資產,流動負債,資產總額,保留盈餘,稅前息前淨利,季底普通股市值,負債總額,營業收入淨額,歸屬母公司淨利（損）,來自營運之現金流量,現金增資(仟股),營業毛利,當季季底P/B,每股淨值(A),當季季底P/E,報酬率
0,2301 光寶科,2013,150430691.0,111356983.0,212361486.0,21463386.0,12228610.0,111260475.0,133124331.0,213214297.0,8754848.0,20688338.0,0.0,30500594.0,1.52,31.80,9.06,NaN
1,2301 光寶科,2014,153720896.0,120288626.0,216533829.0,20959086.0,8421506.0,85236897.0,137361191.0,230631974.0,6460808.0,14131417.0,0.0,28248114.0,1.14,32.38,9.97,-0.236321
2,2301 光寶科,2015,150313451.0,109830689.0,209653863.0,23366328.0,10585356.0,74367457.0,129970270.0,216928734.0,7222899.0,15552262.0,0.0,28141217.0,0.98,32.92,7.40,-0.126015
3,2301 光寶科,2016,157924039.0,116321118.0,210856788.0,27496140.0,13199864.0,114252136.0,131576352.0,229571758.0,9416351.0,14512514.0,0.0,31258268.0,1.50,32.67,8.89,0.518989
4,2301 光寶科,2017,145063307.0,114596707.0,189995757.0,23219598.0,3975438.0,95445200.0,116228328.0,214564322.0,2629334.0,11153180.0,0.0,27709817.0,1.35,30.34,9.54,-0.164187
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
708,9912 偉聯,2020,360049.0,260527.0,691393.0,-18828.0,-34686.0,451432.0,271818.0,538991.0,-38233.0,79243.0,0.0,182143.0,1.08,8.22,NaN,-0.013161
709,9912 偉聯,2021,441796.0,334589.0,764082.0,-587.0,29144.0,562627.0,343497.0,632260.0,18241.0,-47312.0,0.0,215586.0,1.34,8.11,25.02,0.224137
710,9912 偉聯,2022,426643.0,332239.0,770953.0,-14346.0,-24377.0,498968.0,339772.0,594422.0,-13759.0,-33671.0,0.0,188983.0,1.19,8.10,-888.00,-0.113035
711,9912 偉聯,2023,400578.0,308373.0,794605.0,27724.0,-39499.0,596354.0,314116.0,510587.0,-20637.0,68206.0,0.0,199660.0,1.26,9.03,NaN,0.180392


# 移除2013年至2024年內上市或者年份不完整的公司

In [3]:
import pandas as pd

# 定義完整的年份範圍
full_years = set(map(str, range(2013, 2025)))

# 取得每間公司的年份清單
company_years = basic_data.groupby("公司")["年月"].apply(set)

# 找出缺少年份的公司
missing_years = {company: full_years - years for company, years in company_years.items() if full_years - years}

# 列出被刪除的公司與缺少的年份
print(f"原本公司數: {len(company_years)}")

for company, missing in missing_years.items():
    print(f"刪除公司: {company}, 缺少年份: {sorted(missing)}")

# 只保留年份完整的公司
valid_companies = set(company_years.keys()) - set(missing_years.keys())
filtered_data = basic_data[basic_data["公司"].isin(valid_companies)]

# 顯示刪除後的公司數
print(f"刪除後的公司數: {len(valid_companies)}")

display(filtered_data)

原本公司數: 61
刪除公司: 2380 虹光, 缺少年份: ['2024']
刪除公司: 2405 輔信, 缺少年份: ['2024']
刪除公司: 3701 大眾控, 缺少年份: ['2024']
刪除公司: 3712 永崴投控, 缺少年份: ['2013', '2014', '2015', '2016']
刪除公司: 6928 攸泰科技, 缺少年份: ['2013', '2014', '2015', '2016', '2017']
刪除公司: 6933 AMAX-KY, 缺少年份: ['2013', '2014', '2015', '2016', '2017', '2018', '2019']
刪除後的公司數: 55


,公司,年月,流動資產,流動負債,資產總額,保留盈餘,稅前息前淨利,季底普通股市值,負債總額,營業收入淨額,歸屬母公司淨利（損）,來自營運之現金流量,現金增資(仟股),營業毛利,當季季底P/B,每股淨值(A),當季季底P/E,報酬率
0,2301 光寶科,2013,150430691.0,111356983.0,212361486.0,21463386.0,12228610.0,111260475.0,133124331.0,213214297.0,8754848.0,20688338.0,0.0,30500594.0,1.52,31.80,9.06,NaN
1,2301 光寶科,2014,153720896.0,120288626.0,216533829.0,20959086.0,8421506.0,85236897.0,137361191.0,230631974.0,6460808.0,14131417.0,0.0,28248114.0,1.14,32.38,9.97,-0.236321
2,2301 光寶科,2015,150313451.0,109830689.0,209653863.0,23366328.0,10585356.0,74367457.0,129970270.0,216928734.0,7222899.0,15552262.0,0.0,28141217.0,0.98,32.92,7.40,-0.126015
3,2301 光寶科,2016,157924039.0,116321118.0,210856788.0,27496140.0,13199864.0,114252136.0,131576352.0,229571758.0,9416351.0,14512514.0,0.0,31258268.0,1.50,32.67,8.89,0.518989
4,2301 光寶科,2017,145063307.0,114596707.0,189995757.0,23219598.0,3975438.0,95445200.0,116228328.0,214564322.0,2629334.0,11153180.0,0.0,27709817.0,1.35,30.34,9.54,-0.164187
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
708,9912 偉聯,2020,360049.0,260527.0,691393.0,-18828.0,-34686.0,451432.0,271818.0,538991.0,-38233.0,79243.0,0.0,182143.0,1.08,8.22,NaN,-0.013161
709,9912 偉聯,2021,441796.0,334589.0,764082.0,-587.0,29144.0,562627.0,343497.0,632260.0,18241.0,-47312.0,0.0,215586.0,1.34,8.11,25.02,0.224137
710,9912 偉聯,2022,426643.0,332239.0,770953.0,-14346.0,-24377.0,498968.0,339772.0,594422.0,-13759.0,-33671.0,0.0,188983.0,1.19,8.10,-888.00,-0.113035
711,9912 偉聯,2023,400578.0,308373.0,794605.0,27724.0,-39499.0,596354.0,314116.0,510587.0,-20637.0,68206.0,0.0,199660.0,1.26,9.03,NaN,0.180392


# 缺失值處理

In [4]:
import pandas as pd

# 定義需要檢查缺失值的列
columns_to_check = [
    '流動資產', '流動負債', '資產總額', '保留盈餘', '稅前息前淨利', 
    '季底普通股市值', '負債總額', '營業收入淨額', '歸屬母公司淨利（損）', 
    '來自營運之現金流量', '現金增資(仟股)', '營業毛利'
]

# 需要檢查的 2011 年度列
columns_to_check_2011 = [
    '流動資產', '流動負債', '資產總額', '負債總額', '營業毛利', 
    '營業收入淨額', '歸屬母公司淨利（損）'
]

# 取得每間公司的年份清單
company_years = filtered_data.groupby("公司")["年月"].apply(set)

# 定義完整的年份範圍
full_years = set(map(str, range(2013, 2025)))

# 找出缺少資料的公司及缺少的年份
missing_years = {}
missing_columns_details = {}

for company, years in company_years.items():
    for year in years:
        if int(year) == 2013:
            # 2011年只檢查特定列的缺失值
            missing_columns = filtered_data[(filtered_data["公司"] == company) & (filtered_data["年月"] == year)][columns_to_check_2011].isna()
        else:
            # 其他年份檢查全部列的缺失值
            missing_columns = filtered_data[(filtered_data["公司"] == company) & (filtered_data["年月"] == year)][columns_to_check].isna()
        
        # 檢查哪些列是缺失的
        missing_columns_list = missing_columns.columns[missing_columns.any()].tolist()
        
        if missing_columns_list:
            if company not in missing_years:
                missing_years[company] = []
            missing_years[company].append(year)
            missing_columns_details[company] = missing_columns_list

# 列出原本的公司數
print(f"原本公司數: {len(company_years)}")

# 列出缺少資料的公司、年份以及缺失的財報項目
for company, missing in missing_years.items():
    print(f"刪除公司: {company}, 缺少年份: {sorted(missing)}")
    print(f"缺少的財報項目: {missing_columns_details[company]}")

# 只保留有效的公司
valid_companies = set(company_years.keys()) - set(missing_years.keys())
filtered_data = filtered_data[filtered_data["公司"].isin(valid_companies)]

# 顯示刪除後的公司數
print(f"刪除後的公司數: {len(valid_companies)}")

# 顯示過濾後的資料
display(filtered_data)



原本公司數: 55
刪除公司: 6414 樺漢, 缺少年份: ['2024']
缺少的財報項目: ['流動資產', '流動負債', '資產總額', '保留盈餘', '稅前息前淨利', '季底普通股市值', '負債總額', '營業收入淨額', '歸屬母公司淨利（損）', '來自營運之現金流量', '營業毛利']
刪除公司: 6579 研揚, 缺少年份: ['2014', '2015']
缺少的財報項目: ['季底普通股市值']
刪除公司: 6591 動力-KY, 缺少年份: ['2014', '2015']
缺少的財報項目: ['季底普通股市值']
刪除公司: 6669 緯穎, 缺少年份: ['2013', '2014', '2015', '2016']
缺少的財報項目: ['季底普通股市值']
刪除後的公司數: 51


,公司,年月,流動資產,流動負債,資產總額,保留盈餘,稅前息前淨利,季底普通股市值,負債總額,營業收入淨額,歸屬母公司淨利（損）,來自營運之現金流量,現金增資(仟股),營業毛利,當季季底P/B,每股淨值(A),當季季底P/E,報酬率
0,2301 光寶科,2013,150430691.0,111356983.0,212361486.0,21463386.0,12228610.0,111260475.0,133124331.0,213214297.0,8754848.0,20688338.0,0.0,30500594.0,1.52,31.80,9.06,NaN
1,2301 光寶科,2014,153720896.0,120288626.0,216533829.0,20959086.0,8421506.0,85236897.0,137361191.0,230631974.0,6460808.0,14131417.0,0.0,28248114.0,1.14,32.38,9.97,-0.236321
2,2301 光寶科,2015,150313451.0,109830689.0,209653863.0,23366328.0,10585356.0,74367457.0,129970270.0,216928734.0,7222899.0,15552262.0,0.0,28141217.0,0.98,32.92,7.40,-0.126015
3,2301 光寶科,2016,157924039.0,116321118.0,210856788.0,27496140.0,13199864.0,114252136.0,131576352.0,229571758.0,9416351.0,14512514.0,0.0,31258268.0,1.50,32.67,8.89,0.518989
4,2301 光寶科,2017,145063307.0,114596707.0,189995757.0,23219598.0,3975438.0,95445200.0,116228328.0,214564322.0,2629334.0,11153180.0,0.0,27709817.0,1.35,30.34,9.54,-0.164187
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
708,9912 偉聯,2020,360049.0,260527.0,691393.0,-18828.0,-34686.0,451432.0,271818.0,538991.0,-38233.0,79243.0,0.0,182143.0,1.08,8.22,NaN,-0.013161
709,9912 偉聯,2021,441796.0,334589.0,764082.0,-587.0,29144.0,562627.0,343497.0,632260.0,18241.0,-47312.0,0.0,215586.0,1.34,8.11,25.02,0.224137
710,9912 偉聯,2022,426643.0,332239.0,770953.0,-14346.0,-24377.0,498968.0,339772.0,594422.0,-13759.0,-33671.0,0.0,188983.0,1.19,8.10,-888.00,-0.113035
711,9912 偉聯,2023,400578.0,308373.0,794605.0,27724.0,-39499.0,596354.0,314116.0,510587.0,-20637.0,68206.0,0.0,199660.0,1.26,9.03,NaN,0.180392


# 淨值比排序篩股(可以調篩選數量)

In [5]:
import pandas as pd

# 按公司分組，取 2024 年的當季季底P/B，並進行排序
top_30_companies = filtered_data[filtered_data["年月"] == "2024"].groupby("公司")["當季季底P/B"].min().nsmallest(30)

# 取得最小的 30 家公司的名稱
top_30_company_names = top_30_companies.index.tolist()

# 過濾出這些公司在所有年份的資料
filtered_data_top_30 = filtered_data[filtered_data["公司"].isin(top_30_company_names)]

# 顯示過濾後的資料
display(filtered_data_top_30)


,公司,年月,流動資產,流動負債,資產總額,保留盈餘,稅前息前淨利,季底普通股市值,負債總額,營業收入淨額,歸屬母公司淨利（損）,來自營運之現金流量,現金增資(仟股),營業毛利,當季季底P/B,每股淨值(A),當季季底P/E,報酬率
12,2305 全友,2013,1018111.0,758273.0,2075762.0,-806931.0,81587.0,709527.0,842840.0,707029.0,40030.0,129831.0,0.0,267535.0,0.59,5.93,11.04,-0.149821
13,2305 全友,2014,1385627.0,691365.0,2198514.0,-654823.0,178876.0,2149147.0,752877.0,798731.0,153103.0,-135687.0,0.0,317522.0,1.52,6.99,56.66,2.036785
14,2305 全友,2015,1827233.0,1125381.0,3130908.0,-634898.0,70968.0,1104394.0,1246512.0,809073.0,23267.0,-293831.0,0.0,303594.0,0.60,9.14,41.42,-0.483849
15,2305 全友,2016,1419343.0,927124.0,2809916.0,-622171.0,36020.0,1645280.0,1036677.0,715031.0,16002.0,516513.0,0.0,248956.0,0.95,8.58,NaN,0.486324
16,2305 全友,2017,1551153.0,419829.0,2912863.0,-108850.0,763182.0,2054543.0,623124.0,646562.0,515453.0,117949.0,0.0,208611.0,0.91,11.10,52.03,0.239234
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
708,9912 偉聯,2020,360049.0,260527.0,691393.0,-18828.0,-34686.0,451432.0,271818.0,538991.0,-38233.0,79243.0,0.0,182143.0,1.08,8.22,NaN,-0.013161
709,9912 偉聯,2021,441796.0,334589.0,764082.0,-587.0,29144.0,562627.0,343497.0,632260.0,18241.0,-47312.0,0.0,215586.0,1.34,8.11,25.02,0.224137
710,9912 偉聯,2022,426643.0,332239.0,770953.0,-14346.0,-24377.0,498968.0,339772.0,594422.0,-13759.0,-33671.0,0.0,188983.0,1.19,8.10,-888.00,-0.113035
711,9912 偉聯,2023,400578.0,308373.0,794605.0,27724.0,-39499.0,596354.0,314116.0,510587.0,-20637.0,68206.0,0.0,199660.0,1.26,9.03,NaN,0.180392


# ZF分數分割

In [6]:
# 第一組列名稱
columns_group_1 = [
    '公司', '年月', '流動資產', '流動負債', '資產總額', '保留盈餘', '稅前息前淨利', 
    '季底普通股市值', '負債總額', '營業收入淨額',"報酬率","當季季底P/B","當季季底P/E"
]

# 第二組列名稱
columns_group_2 = [
    '公司', '年月', '流動資產', '流動負債', '資產總額', '負債總額', '營業收入淨額', 
    '歸屬母公司淨利（損）', '來自營運之現金流量', '現金增資(仟股)', '營業毛利',"報酬率","當季季底P/B","當季季底P/E"
]

# 分成兩組資料
z_score_test = filtered_data[columns_group_1]
f_score_test = filtered_data[columns_group_2]

# 顯示兩組資料
display(z_score_test)
display(f_score_test)


,公司,年月,流動資產,流動負債,資產總額,保留盈餘,稅前息前淨利,季底普通股市值,負債總額,營業收入淨額,報酬率,當季季底P/B,當季季底P/E
0,2301 光寶科,2013,150430691.0,111356983.0,212361486.0,21463386.0,12228610.0,111260475.0,133124331.0,213214297.0,NaN,1.52,9.06
1,2301 光寶科,2014,153720896.0,120288626.0,216533829.0,20959086.0,8421506.0,85236897.0,137361191.0,230631974.0,-0.236321,1.14,9.97
2,2301 光寶科,2015,150313451.0,109830689.0,209653863.0,23366328.0,10585356.0,74367457.0,129970270.0,216928734.0,-0.126015,0.98,7.40
3,2301 光寶科,2016,157924039.0,116321118.0,210856788.0,27496140.0,13199864.0,114252136.0,131576352.0,229571758.0,0.518989,1.50,8.89
4,2301 光寶科,2017,145063307.0,114596707.0,189995757.0,23219598.0,3975438.0,95445200.0,116228328.0,214564322.0,-0.164187,1.35,9.54
...,...,...,...,...,...,...,...,...,...,...,...,...,...
708,9912 偉聯,2020,360049.0,260527.0,691393.0,-18828.0,-34686.0,451432.0,271818.0,538991.0,-0.013161,1.08,NaN
709,9912 偉聯,2021,441796.0,334589.0,764082.0,-587.0,29144.0,562627.0,343497.0,632260.0,0.224137,1.34,25.02
710,9912 偉聯,2022,426643.0,332239.0,770953.0,-14346.0,-24377.0,498968.0,339772.0,594422.0,-0.113035,1.19,-888.00
711,9912 偉聯,2023,400578.0,308373.0,794605.0,27724.0,-39499.0,596354.0,314116.0,510587.0,0.180392,1.26,NaN


,公司,年月,流動資產,流動負債,資產總額,負債總額,營業收入淨額,歸屬母公司淨利（損）,來自營運之現金流量,現金增資(仟股),營業毛利,報酬率,當季季底P/B,當季季底P/E
0,2301 光寶科,2013,150430691.0,111356983.0,212361486.0,133124331.0,213214297.0,8754848.0,20688338.0,0.0,30500594.0,NaN,1.52,9.06
1,2301 光寶科,2014,153720896.0,120288626.0,216533829.0,137361191.0,230631974.0,6460808.0,14131417.0,0.0,28248114.0,-0.236321,1.14,9.97
2,2301 光寶科,2015,150313451.0,109830689.0,209653863.0,129970270.0,216928734.0,7222899.0,15552262.0,0.0,28141217.0,-0.126015,0.98,7.40
3,2301 光寶科,2016,157924039.0,116321118.0,210856788.0,131576352.0,229571758.0,9416351.0,14512514.0,0.0,31258268.0,0.518989,1.50,8.89
4,2301 光寶科,2017,145063307.0,114596707.0,189995757.0,116228328.0,214564322.0,2629334.0,11153180.0,0.0,27709817.0,-0.164187,1.35,9.54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
708,9912 偉聯,2020,360049.0,260527.0,691393.0,271818.0,538991.0,-38233.0,79243.0,0.0,182143.0,-0.013161,1.08,NaN
709,9912 偉聯,2021,441796.0,334589.0,764082.0,343497.0,632260.0,18241.0,-47312.0,0.0,215586.0,0.224137,1.34,25.02
710,9912 偉聯,2022,426643.0,332239.0,770953.0,339772.0,594422.0,-13759.0,-33671.0,0.0,188983.0,-0.113035,1.19,-888.00
711,9912 偉聯,2023,400578.0,308373.0,794605.0,314116.0,510587.0,-20637.0,68206.0,0.0,199660.0,0.180392,1.26,NaN


# Z分數計算

In [7]:
z_score_test=z_score_test.copy()
z_score_test.loc[:, 'Z1'] = (z_score_test['流動資產'] - z_score_test['流動負債']) / z_score_test['資產總額']
z_score_test.loc[:, 'Z2'] = z_score_test['保留盈餘'] / z_score_test['資產總額']
z_score_test.loc[:, 'Z3'] = z_score_test['稅前息前淨利'] / z_score_test['資產總額']
z_score_test.loc[:, 'Z4'] = z_score_test['季底普通股市值'] / z_score_test['負債總額']
z_score_test.loc[:, 'Z5'] = z_score_test['營業收入淨額'] / z_score_test['資產總額']

# 計算 Z 分數
z_score_test['Z分數'] = (
    z_score_test['Z1'] * 1.2 + 
    z_score_test['Z2'] * 1.4 + 
    z_score_test['Z3'] * 3.3 + 
    z_score_test['Z4'] * 0.6 + 
    z_score_test['Z5'] * 0.999
)


display(z_score_test)





,公司,年月,流動資產,流動負債,資產總額,保留盈餘,稅前息前淨利,季底普通股市值,負債總額,營業收入淨額,報酬率,當季季底P/B,當季季底P/E,Z1,Z2,Z3,Z4,Z5,Z分數
0,2301 光寶科,2013,150430691.0,111356983.0,212361486.0,21463386.0,12228610.0,111260475.0,133124331.0,213214297.0,NaN,1.52,9.06,0.183996,0.101070,0.057584,0.835764,1.004016,2.056791
1,2301 光寶科,2014,153720896.0,120288626.0,216533829.0,20959086.0,8421506.0,85236897.0,137361191.0,230631974.0,-0.236321,1.14,9.97,0.154397,0.096794,0.038892,0.620531,1.065108,1.885495
2,2301 光寶科,2015,150313451.0,109830689.0,209653863.0,23366328.0,10585356.0,74367457.0,129970270.0,216928734.0,-0.126015,0.98,7.40,0.193093,0.111452,0.050490,0.572188,1.034699,1.931338
3,2301 光寶科,2016,157924039.0,116321118.0,210856788.0,27496140.0,13199864.0,114252136.0,131576352.0,229571758.0,0.518989,1.50,8.89,0.197304,0.130402,0.062601,0.868333,1.088757,2.234579
4,2301 光寶科,2017,145063307.0,114596707.0,189995757.0,23219598.0,3975438.0,95445200.0,116228328.0,214564322.0,-0.164187,1.35,9.54,0.160354,0.122211,0.020924,0.821187,1.129311,2.053463
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
708,9912 偉聯,2020,360049.0,260527.0,691393.0,-18828.0,-34686.0,451432.0,271818.0,538991.0,-0.013161,1.08,NaN,0.143944,-0.027232,-0.050168,1.660788,0.779573,1.744319
709,9912 偉聯,2021,441796.0,334589.0,764082.0,-587.0,29144.0,562627.0,343497.0,632260.0,0.224137,1.34,25.02,0.140308,-0.000768,0.038143,1.637939,0.827477,2.102577
710,9912 偉聯,2022,426643.0,332239.0,770953.0,-14346.0,-24377.0,498968.0,339772.0,594422.0,-0.113035,1.19,-888.00,0.122451,-0.018608,-0.031619,1.468538,0.771022,1.667920
711,9912 偉聯,2023,400578.0,308373.0,794605.0,27724.0,-39499.0,596354.0,314116.0,510587.0,0.180392,1.26,NaN,0.116039,0.034890,-0.049709,1.898515,0.642567,1.805087


# Z分數資料處理

In [8]:
z_score=z_score_test.copy()

# 先轉換 '年月' 為整數
z_score['年月'] = z_score['年月'].astype(int)

# 刪除 2013 年的資料
z_score = z_score[z_score['年月'] > 2013].copy()

display(z_score)

z_score_return=z_score.copy()

z_score_return = z_score_return[["公司", "年月", "報酬率", "Z分數","當季季底P/B","當季季底P/E"]]

# 針對每個公司，將 "報酬率" 欄位往下移動一格
z_score_return["明年報酬率"] = z_score_return.groupby("公司")["報酬率"].shift(-1)


display(z_score_return)




,公司,年月,流動資產,流動負債,資產總額,保留盈餘,稅前息前淨利,季底普通股市值,負債總額,營業收入淨額,報酬率,當季季底P/B,當季季底P/E,Z1,Z2,Z3,Z4,Z5,Z分數
1,2301 光寶科,2014,153720896.0,120288626.0,216533829.0,20959086.0,8421506.0,85236897.0,137361191.0,230631974.0,-0.236321,1.14,9.97,0.154397,0.096794,0.038892,0.620531,1.065108,1.885495
2,2301 光寶科,2015,150313451.0,109830689.0,209653863.0,23366328.0,10585356.0,74367457.0,129970270.0,216928734.0,-0.126015,0.98,7.40,0.193093,0.111452,0.050490,0.572188,1.034699,1.931338
3,2301 光寶科,2016,157924039.0,116321118.0,210856788.0,27496140.0,13199864.0,114252136.0,131576352.0,229571758.0,0.518989,1.50,8.89,0.197304,0.130402,0.062601,0.868333,1.088757,2.234579
4,2301 光寶科,2017,145063307.0,114596707.0,189995757.0,23219598.0,3975438.0,95445200.0,116228328.0,214564322.0,-0.164187,1.35,9.54,0.160354,0.122211,0.020924,0.821187,1.129311,2.053463
5,2301 光寶科,2018,157984918.0,120612081.0,197136069.0,30545001.0,11659439.0,95445200.0,122457668.0,207109088.0,0.004043,1.34,7.38,0.189579,0.154944,0.059144,0.779414,1.050590,2.156779
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
708,9912 偉聯,2020,360049.0,260527.0,691393.0,-18828.0,-34686.0,451432.0,271818.0,538991.0,-0.013161,1.08,NaN,0.143944,-0.027232,-0.050168,1.660788,0.779573,1.744319
709,9912 偉聯,2021,441796.0,334589.0,764082.0,-587.0,29144.0,562627.0,343497.0,632260.0,0.224137,1.34,25.02,0.140308,-0.000768,0.038143,1.637939,0.827477,2.102577
710,9912 偉聯,2022,426643.0,332239.0,770953.0,-14346.0,-24377.0,498968.0,339772.0,594422.0,-0.113035,1.19,-888.00,0.122451,-0.018608,-0.031619,1.468538,0.771022,1.667920
711,9912 偉聯,2023,400578.0,308373.0,794605.0,27724.0,-39499.0,596354.0,314116.0,510587.0,0.180392,1.26,NaN,0.116039,0.034890,-0.049709,1.898515,0.642567,1.805087


,公司,年月,報酬率,Z分數,當季季底P/B,當季季底P/E,明年報酬率
1,2301 光寶科,2014,-0.236321,1.885495,1.14,9.97,-0.126015
2,2301 光寶科,2015,-0.126015,1.931338,0.98,7.40,0.518989
3,2301 光寶科,2016,0.518989,2.234579,1.50,8.89,-0.164187
4,2301 光寶科,2017,-0.164187,2.053463,1.35,9.54,0.004043
5,2301 光寶科,2018,0.004043,2.156779,1.34,7.38,0.213094
...,...,...,...,...,...,...,...
708,9912 偉聯,2020,-0.013161,1.744319,1.08,NaN,0.224137
709,9912 偉聯,2021,0.224137,2.102577,1.34,25.02,-0.113035
710,9912 偉聯,2022,-0.113035,1.667920,1.19,-888.00,0.180392
711,9912 偉聯,2023,0.180392,1.805087,1.26,NaN,0.056821


# F分數計算

In [9]:
f_score_test = f_score_test.copy()  # 創建副本，避免警告

# 確保數據按公司 & 年份排序
f_score_test = f_score_test.sort_values(by=['公司', '年月']).copy()

# 計算 F1 ~ F9
f_score_test['F1'] = f_score_test['歸屬母公司淨利（損）'] / f_score_test['資產總額']
f_score_test['F2'] = f_score_test['F1'] - f_score_test.groupby('公司')['F1'].shift(1)
f_score_test['F3'] = f_score_test['來自營運之現金流量'] / f_score_test['資產總額']
f_score_test['F4'] = f_score_test['F1'] - f_score_test['F3']

f_score_test['F5'] = (f_score_test['負債總額'] / f_score_test['資產總額']) - \
                (f_score_test.groupby('公司')['負債總額'].shift(1) / f_score_test.groupby('公司')['資產總額'].shift(1))

f_score_test['F6'] = (f_score_test['流動資產'] / f_score_test['流動負債']) - \
                (f_score_test.groupby('公司')['流動資產'].shift(1) / f_score_test.groupby('公司')['流動負債'].shift(1))

f_score_test['F7'] = (f_score_test['現金增資(仟股)'] != 0).astype(int)

f_score_test['F8'] = (f_score_test['營業毛利'] / f_score_test['營業收入淨額']) - \
                (f_score_test.groupby('公司')['營業毛利'].shift(1) / f_score_test.groupby('公司')['營業收入淨額'].shift(1))

f_score_test['F9'] = (f_score_test['營業收入淨額'] / f_score_test['資產總額']) - \
                (f_score_test.groupby('公司')['營業收入淨額'].shift(1) / f_score_test.groupby('公司')['資產總額'].shift(1))



# 需要轉換的欄位
cols_to_convert = ["F1","F2","F3","F6","F8","F9"]
cols = ["F4","F5","F7"]

# 將正數轉為 1，負數轉為 0
f_score_test[cols_to_convert] = f_score_test[cols_to_convert].map(lambda x: 1 if x > 0 else 0)

# 將正數轉為 0，負數轉為 1
f_score_test[cols] = f_score_test[cols].map(lambda x: 0 if x > 0 else 1)

# 計算 F1 到 F9 的總和
f_score_test["F分數"] = f_score_test[["F1", "F2", "F3", "F4", "F5", "F6", "F7", "F8", "F9"]].sum(axis=1)

display(f_score_test)


,公司,年月,流動資產,流動負債,資產總額,負債總額,營業收入淨額,歸屬母公司淨利（損）,來自營運之現金流量,現金增資(仟股),...,F1,F2,F3,F4,F5,F6,F7,F8,F9,F分數
0,2301 光寶科,2013,150430691.0,111356983.0,212361486.0,133124331.0,213214297.0,8754848.0,20688338.0,0.0,...,1,0,1,1,1,0,1,0,0,5
1,2301 光寶科,2014,153720896.0,120288626.0,216533829.0,137361191.0,230631974.0,6460808.0,14131417.0,0.0,...,1,0,1,1,0,0,1,0,1,5
2,2301 光寶科,2015,150313451.0,109830689.0,209653863.0,129970270.0,216928734.0,7222899.0,15552262.0,0.0,...,1,1,1,1,1,1,1,1,0,8
3,2301 光寶科,2016,157924039.0,116321118.0,210856788.0,131576352.0,229571758.0,9416351.0,14512514.0,0.0,...,1,1,1,1,0,0,1,1,1,7
4,2301 光寶科,2017,145063307.0,114596707.0,189995757.0,116228328.0,214564322.0,2629334.0,11153180.0,0.0,...,1,0,1,1,1,0,1,0,1,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
708,9912 偉聯,2020,360049.0,260527.0,691393.0,271818.0,538991.0,-38233.0,79243.0,0.0,...,0,1,1,1,1,1,1,1,1,8
709,9912 偉聯,2021,441796.0,334589.0,764082.0,343497.0,632260.0,18241.0,-47312.0,0.0,...,1,1,0,0,0,0,1,1,1,5
710,9912 偉聯,2022,426643.0,332239.0,770953.0,339772.0,594422.0,-13759.0,-33671.0,0.0,...,0,0,0,0,1,0,1,0,0,2
711,9912 偉聯,2023,400578.0,308373.0,794605.0,314116.0,510587.0,-20637.0,68206.0,0.0,...,0,0,1,1,1,1,1,1,0,6


# F分數資料處理

In [10]:
f_score=f_score_test.copy()

# 先轉換 '年月' 為整數
f_score['年月'] = f_score['年月'].astype(int)

# 刪除 2013 年的資料
f_score = f_score[f_score['年月'] > 2013].copy()

# 重置索引
f_score.reset_index(drop=True, inplace=True)

# 顯示處理後的 DataFrame
display(f_score)


f_score_return=f_score.copy()

f_score_return = f_score_return[["公司", "年月", "報酬率", "F分數","當季季底P/B","當季季底P/E"]]

# 針對每個公司，將 "報酬率" 欄位往下移動一格
f_score_return["明年報酬率"] = f_score_return.groupby("公司")["報酬率"].shift(-1)


display(f_score_return)

,公司,年月,流動資產,流動負債,資產總額,負債總額,營業收入淨額,歸屬母公司淨利（損）,來自營運之現金流量,現金增資(仟股),...,F1,F2,F3,F4,F5,F6,F7,F8,F9,F分數
0,2301 光寶科,2014,153720896.0,120288626.0,216533829.0,137361191.0,230631974.0,6460808.0,14131417.0,0.0,...,1,0,1,1,0,0,1,0,1,5
1,2301 光寶科,2015,150313451.0,109830689.0,209653863.0,129970270.0,216928734.0,7222899.0,15552262.0,0.0,...,1,1,1,1,1,1,1,1,0,8
2,2301 光寶科,2016,157924039.0,116321118.0,210856788.0,131576352.0,229571758.0,9416351.0,14512514.0,0.0,...,1,1,1,1,0,0,1,1,1,7
3,2301 光寶科,2017,145063307.0,114596707.0,189995757.0,116228328.0,214564322.0,2629334.0,11153180.0,0.0,...,1,0,1,1,1,0,1,0,1,6
4,2301 光寶科,2018,157984918.0,120612081.0,197136069.0,122457668.0,207109088.0,7956838.0,13483544.0,0.0,...,1,1,1,1,0,1,1,1,0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
556,9912 偉聯,2020,360049.0,260527.0,691393.0,271818.0,538991.0,-38233.0,79243.0,0.0,...,0,1,1,1,1,1,1,1,1,8
557,9912 偉聯,2021,441796.0,334589.0,764082.0,343497.0,632260.0,18241.0,-47312.0,0.0,...,1,1,0,0,0,0,1,1,1,5
558,9912 偉聯,2022,426643.0,332239.0,770953.0,339772.0,594422.0,-13759.0,-33671.0,0.0,...,0,0,0,0,1,0,1,0,0,2
559,9912 偉聯,2023,400578.0,308373.0,794605.0,314116.0,510587.0,-20637.0,68206.0,0.0,...,0,0,1,1,1,1,1,1,0,6


,公司,年月,報酬率,F分數,當季季底P/B,當季季底P/E,明年報酬率
0,2301 光寶科,2014,-0.236321,5,1.14,9.97,-0.126015
1,2301 光寶科,2015,-0.126015,8,0.98,7.40,0.518989
2,2301 光寶科,2016,0.518989,7,1.50,8.89,-0.164187
3,2301 光寶科,2017,-0.164187,6,1.35,9.54,0.004043
4,2301 光寶科,2018,0.004043,7,1.34,7.38,0.213094
...,...,...,...,...,...,...,...
556,9912 偉聯,2020,-0.013161,8,1.08,NaN,0.224137
557,9912 偉聯,2021,0.224137,5,1.34,25.02,-0.113035
558,9912 偉聯,2022,-0.113035,2,1.19,-888.00,0.180392
559,9912 偉聯,2023,0.180392,6,1.26,NaN,0.056821


# ZF分數最終表格

In [11]:
# 合併 f_score_return 和 z_score_return，依據 '公司' 和 '年月'
merged_df = pd.merge(f_score_return, z_score_return, on=['公司', '年月', '報酬率', '明年報酬率',"當季季底P/B","當季季底P/E"], how='inner')

# 重新排列列名稱順序
merged_zf_score = merged_df[['公司', '年月', '報酬率', '明年報酬率', 'F分數', 'Z分數',"當季季底P/B","當季季底P/E"]]

# 顯示結果
display(merged_zf_score)


,公司,年月,報酬率,明年報酬率,F分數,Z分數,當季季底P/B,當季季底P/E
0,2301 光寶科,2014,-0.236321,-0.126015,5,1.885495,1.14,9.97
1,2301 光寶科,2015,-0.126015,0.518989,8,1.931338,0.98,7.40
2,2301 光寶科,2016,0.518989,-0.164187,7,2.234579,1.50,8.89
3,2301 光寶科,2017,-0.164187,0.004043,6,2.053463,1.35,9.54
4,2301 光寶科,2018,0.004043,0.213094,7,2.156779,1.34,7.38
...,...,...,...,...,...,...,...,...
556,9912 偉聯,2020,-0.013161,0.224137,8,1.744319,1.08,NaN
557,9912 偉聯,2021,0.224137,-0.113035,5,2.102577,1.34,25.02
558,9912 偉聯,2022,-0.113035,0.180392,2,1.667920,1.19,-888.00
559,9912 偉聯,2023,0.180392,0.056821,6,1.805087,1.26,NaN


# 相關係數矩陣

In [12]:

# 計算每個公司的相關係數
correlation_zf_score = merged_zf_score.groupby("公司")[[ '報酬率', '明年報酬率', 'F分數', 'Z分數']].corr()
correlation_zf_score

報酬率     明年報酬率       F分數       Z分數
公司                                                    
2301 光寶科 報酬率    1.000000 -0.445423  0.458712  0.652467
         明年報酬率 -0.445423  1.000000  0.302869 -0.125496
         F分數    0.458712  0.302869  1.000000  0.043544
         Z分數    0.652467 -0.125496  0.043544  1.000000
2305 全友  報酬率    1.000000 -0.442578  0.064409  0.160528
...                  ...       ...       ...       ...
8210 勤誠  Z分數    0.452822 -0.539719  0.409344  1.000000
9912 偉聯  報酬率    1.000000  0.282046  0.332080 -0.266860
         明年報酬率  0.282046  1.000000  0.270651 -0.645806
         F分數    0.332080  0.270651  1.000000  0.365977
         Z分數   -0.266860 -0.645806  0.365977  1.000000

[204 rows x 4 columns]

# 趨勢圖函數 (輸入股票代號及名稱)

In [13]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import openpyxl
from io import BytesIO

# 設定字體（適用 Windows，如為 Mac/Linux 可改成 Heiti TC 或 Noto Sans CJK）
plt.rcParams['font.family'] = 'Microsoft JhengHei'
plt.rcParams['axes.unicode_minus'] = False  # 避免負號變成方塊

# 設定圖片存放資料夾
img_dir = r"E:\碩士\財務工程(二)\圖片檔"
os.makedirs(img_dir, exist_ok=True)  # 若資料夾不存在則建立

# 創建圖片並插入到 Excel，並儲存到資料夾
def plot_and_insert_images(df, z_score_companies, f_score_companies, worksheet):
    row_offset = 1  # 記錄當前圖片要插入的 Excel 行數

    # 畫 Z 分數的公司
    for company_name in z_score_companies:
        company_data = df[df['公司'] == company_name].copy()
        company_data = company_data.dropna(subset=['Z分數'])  # 只篩選有 Z 分數的
        company_data['年月'] = company_data['年月'].astype(str)  

        # 設置畫布大小
        fig, ax1 = plt.subplots(figsize=(8, 4))

        # 繪製 Z分數 vs 明年報酬率
        ax1.plot(company_data['年月'], company_data['Z分數'], marker='o', label='Z分數', color='green')
        ax1.set_xlabel('年月')
        ax1.set_ylabel('Z分數', color='green')
        ax1.tick_params(axis='y', labelcolor='green')
        ax1.set_title(f'{company_name} - Z分數 vs 明年報酬率')

        ax2 = ax1.twinx()
        ax2.plot(company_data['年月'], company_data['明年報酬率'], marker='x', label='明年報酬率', color='red')
        ax2.set_ylabel('明年報酬率', color='red')
        ax2.tick_params(axis='y', labelcolor='red')

        plt.xticks(rotation=45)  
        plt.tight_layout()

        # **儲存圖片到本地資料夾**
        img_path = os.path.join(img_dir, f"Z_score_{company_name}.png")
        plt.savefig(img_path, format='png', dpi=100)  # 存成 PNG 檔案

        # **儲存圖片至 BytesIO**
        img_stream = BytesIO()
        plt.savefig(img_stream, format='png', dpi=100)  
        img_stream.seek(0)
        plt.close()  

        # 插入圖片到 Excel
        img = openpyxl.drawing.image.Image(img_stream)
        img.width, img.height = 800, 300  
        worksheet.add_image(img, f"A{row_offset}")  
        row_offset += 17  # 間隔 17 行，避免圖片重疊

    # 畫 F 分數的公司
    for company_name in f_score_companies:
        company_data = df[df['公司'] == company_name].copy()
        company_data = company_data.dropna(subset=['F分數'])  # 只篩選有 F 分數的
        company_data['年月'] = company_data['年月'].astype(str)

        # 設置畫布大小
        fig, ax1 = plt.subplots(figsize=(8, 4))

        # 繪製 F分數 vs 明年報酬率
        ax1.plot(company_data['年月'], company_data['F分數'], marker='o', label='F分數', color='blue')
        ax1.set_xlabel('年月')
        ax1.set_ylabel('F分數', color='blue')
        ax1.tick_params(axis='y', labelcolor='blue')
        ax1.set_title(f'{company_name} - F分數 vs 明年報酬率')

        ax2 = ax1.twinx()
        ax2.plot(company_data['年月'], company_data['明年報酬率'], marker='x', label='明年報酬率', color='red')
        ax2.set_ylabel('明年報酬率', color='red')
        ax2.tick_params(axis='y', labelcolor='red')

        plt.xticks(rotation=45)  
        plt.tight_layout()

        # **儲存圖片到本地資料夾**
        img_path = os.path.join(img_dir, f"F_score_{company_name}.png")
        plt.savefig(img_path, format='png', dpi=100)  # 存成 PNG 檔案

        # **儲存圖片至 BytesIO**
        img_stream = BytesIO()
        plt.savefig(img_stream, format='png', dpi=100)  
        img_stream.seek(0)
        plt.close()

        # 插入圖片到 Excel
        img = openpyxl.drawing.image.Image(img_stream)
        img.width, img.height = 800, 300  
        worksheet.add_image(img, f"A{row_offset}")  
        row_offset += 17  # 間隔 17 行，避免圖片重疊

# 創建新的 Excel 並寫入數據 & 圖片
def save_to_new_excel(excel_file, df, z_score_companies, f_score_companies, z_score, z_score_return, f_score, f_score_return, correlation_zf_score):
    # **使用 mode='w' 新建 Excel 檔案**
    with pd.ExcelWriter(excel_file, engine='openpyxl', mode='w') as writer:
        df.to_excel(writer, sheet_name="merged_zf_score", index=False)

        workbook = writer.book
        worksheet = workbook.create_sheet("分數與報酬率趨勢圖")  # 創建新的圖片工作表

        # 插入圖片
        plot_and_insert_images(df, z_score_companies, f_score_companies, worksheet)

        # 寫入其他數據表
        z_score.to_excel(writer, sheet_name="z_score", index=False)
        z_score_return.to_excel(writer, sheet_name="z_score_Return", index=False)
        f_score.to_excel(writer, sheet_name="f_Score", index=False)
        f_score_return.to_excel(writer, sheet_name="f_score_Return", index=False)
        correlation_zf_score.reset_index().to_excel(writer, sheet_name="correlation_zf_score", index=False)

# 設定新的 Excel 檔案路徑
new_excel_file = r"E:\碩士\財務工程(二)\ZF_score_PE5.xlsx"

# **分開 Z 分數 & F 分數的公司**
z_score_companies = ['2365 昆盈','2395 研華','6206 飛捷','2399 映泰','3416 融程電','2331 精英','8163 達方','9912 偉聯','2362 藍天','3494 誠研']
f_score_companies = ['2305 全友','2364 倫飛','2365 昆盈','6117 迎廣','8210 勤誠','3002 歐格','3057 喬鼎','2465 麗臺','5258 虹堡','9912 偉聯','6206 飛捷','2399 映泰']

# **執行新建 Excel 並儲存**
save_to_new_excel(new_excel_file, merged_zf_score, z_score_companies, f_score_companies, z_score, z_score_return, f_score, f_score_return, correlation_zf_score)

print(f"新 Excel 檔案已成功儲存！\n檔案路徑：{new_excel_file}\n圖片儲存資料夾：{img_dir}")


新 Excel 檔案已成功儲存！
檔案路徑：E:\碩士\財務工程(二)\ZF_score_PE5.xlsx
圖片儲存資料夾：E:\碩士\財務工程(二)\圖片檔


# 股價走勢圖

In [14]:
import os
import yfinance as yf
import matplotlib.pyplot as plt
import openpyxl
from io import BytesIO

# 定義股票代號
tickers = ["6206.TW", "2399.TW"]

# 下載近 5 年的股價
start_date = "2019-01-01"
end_date = "2024-03-22"
fiti = yf.download("6206.TW", start=start_date, end=end_date, auto_adjust=False)
acepill = yf.download("2399.TW", start=start_date, end=end_date, auto_adjust=False)

# 設定中文字體
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei']  # Windows 預設中文
plt.rcParams['axes.unicode_minus'] = False  # 確保負號能正確顯示

# 繪製股價走勢圖
plt.figure(figsize=(14, 7))
plt.plot(fiti.index, fiti["Close"], label="飛捷科技 (6206.TW)", color="blue")
plt.plot(acepill.index, acepill["Close"], label="映泰 (2399.TW)", color="red")
plt.xlabel("日期")
plt.ylabel("收盤價 (新台幣)")
plt.title("飛捷科技 & 映泰 過去五年股價走勢")
plt.legend()
plt.grid(True)

# 指定儲存圖片的資料夾
image_folder = r"E:\碩士\財務工程(二)\圖片檔"
os.makedirs(image_folder, exist_ok=True)  # 若資料夾不存在則創建
image_path = f"{image_folder}\\股價走勢圖.png"

# 儲存圖片到指定資料夾
plt.savefig(image_path, format='png', dpi=300)  # 高解析度保存

# 保存圖片至 BytesIO 以插入 Excel
img_stream = BytesIO()
plt.savefig(img_stream, format='png', dpi=100)  # 降低 DPI 以適應 Excel
img_stream.seek(0)
plt.close()  # 關閉圖表，避免記憶體占用

# 指定 Excel 檔案路徑
excel_file = r"E:\碩士\財務工程(二)\ZF_score_PE5.xlsx"

# 使用 openpyxl 讀取已存在的 Excel 檔案
wb = openpyxl.load_workbook(excel_file)

# 創建新的工作表
ws = wb.create_sheet("股價走勢圖")

# 插入圖片到新工作表
img = openpyxl.drawing.image.Image(img_stream)
img.width, img.height = 800, 400  # 設定圖片大小
ws.add_image(img, "A1")  # 插入圖片，A1 表示放在單元格 A1

# 保存修改後的 Excel 檔案
wb.save(excel_file)

print(f"圖片已成功插入 Excel，並儲存到資料夾：{image_path}")


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


圖片已成功插入 Excel，並儲存到資料夾：E:\碩士\財務工程(二)\圖片檔\股價走勢圖.png


# 推薦公司營收圖 Goodinfo

In [15]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter, YearLocator
import openpyxl
from io import BytesIO
import matplotlib
from matplotlib import font_manager

# 設定中文字型 (根據您系統的字型調整)
font_path = 'C:/Windows/Fonts/msyh.ttc'  # 微軟正黑體（根據您的系統路徑調整）
font_prop = font_manager.FontProperties(fname=font_path)


# 檔案路徑
file_paths = [
    r"E:\碩士\財務工程(二)\飛捷.xlsx",
    r"E:\碩士\財務工程(二)\映泰.xlsx"
]

# 設定新的 Excel 檔案路徑
file_path4 = r"E:\碩士\財務工程(二)\ZF_score_PE5.xlsx"

# 目標資料夾，用來儲存圖片
image_folder = r"E:\碩士\財務工程(二)\圖片檔"

# 確保圖片資料夾存在
os.makedirs(image_folder, exist_ok=True)


# 讀取 Excel 檔案並創建圖表並插入圖片

def plot_and_insert_images(df, file_path, worksheet, start_row):
    # 選擇營收 (億) 欄位
    revenue_col = [col for col in df.columns if "營收 (億)" in col][0]  # 找到第一個「營收 (億)」

    # 將選擇的欄位轉換為數字
    df[revenue_col] = pd.to_numeric(df[revenue_col], errors="coerce")

    # 將「月別」欄位轉換為日期格式
    df['月別'] = pd.to_datetime(df['月別'], errors='coerce')

    # **圖表 1：營收 (億) 變化趨勢**
    plt.figure(figsize=(10, 5))
    plt.plot(df['月別'], df[revenue_col], marker="o", linestyle="-", color="b", label="營收 (億)")
    plt.title(f"營收 (億) 變化趨勢 - {file_path}")
    plt.xlabel("年份")
    plt.ylabel("營收 (億)")
    plt.gca().xaxis.set_major_locator(YearLocator())  # 每年顯示一次
    plt.gca().xaxis.set_major_formatter(DateFormatter('%Y'))  # 顯示年份（%Y）
    plt.xticks(rotation=45)
    plt.legend()
    plt.grid(True)

        # 保存圖片到指定資料夾
    image_file = os.path.join(image_folder, f"{os.path.basename(file_path)}_growth.png")
    plt.savefig(image_file, format='png', dpi=100)

    # 保存圖片至 BytesIO
    img_stream = BytesIO()
    plt.savefig(img_stream, format='png', dpi=100)  # 降低 DPI 以適應 Excel
    img_stream.seek(0)
    plt.close()  # 關閉圖表，避免記憶體占用

    # 插入圖片到 Excel
    img = openpyxl.drawing.image.Image(img_stream)
    img.width, img.height = 800, 400  # 設定圖片大小
    worksheet.add_image(img, f"A{start_row}")  # 插入圖片，動態設定單元格位置

# 檔案處理與插入圖片
def save_to_excel_with_images(file_paths, file_path4):
    # 使用 openpyxl 加載現有的 Excel 文件
    wb = openpyxl.load_workbook(file_path4)
    worksheet = wb.create_sheet("推薦公司圖表營收")  # 新建一個名為 "圖表" 的工作表

    start_row = 1  # 設定第一張圖表的起始行數

    # 處理每個原始 Excel 檔案
    for file_path in file_paths:
        df = pd.read_excel(file_path, header=None, dtype=str)  # 讀取 Excel，不設定標題

        # 合併前 4 行作為標題，去除 nan
        df.columns = df.iloc[:4].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=0)
        df = df[4:].reset_index(drop=True)  # 刪除前 4 行

        # 去除重複欄位名稱
        df = df.loc[:, ~df.columns.duplicated()]

        # 插入圖表
        plot_and_insert_images(df, file_path, worksheet, start_row)

        # 更新下一個圖表的起始行數
        start_row += 40  # 每個圖表間隔 45 行

    # 保存修改後的 Excel 檔案
    wb.save(file_path4)
    print(f"圖片已成功插入並儲存到 {file_path4}！")
save_to_excel_with_images(file_paths, file_path4)


圖片已成功插入並儲存到 E:\碩士\財務工程(二)\ZF_score_PE5.xlsx！


# 推薦公司營收變動圖

In [16]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter, YearLocator
import openpyxl
import os
from matplotlib import font_manager

# 設定中文字型 (根據您系統的字型調整)
font_path = 'C:/Windows/Fonts/msyh.ttc'  # 微軟正黑體（根據您的系統路徑調整）
font_prop = font_manager.FontProperties(fname=font_path)

# 檔案路徑
file_paths = [
    r"E:\碩士\財務工程(二)\飛捷.xlsx",
    r"E:\碩士\財務工程(二)\映泰.xlsx"
]

# 設定新的 Excel 檔案路徑
file_path4 = r"E:\碩士\財務工程(二)\ZF_score_PE5.xlsx"

# 目標資料夾，用來儲存圖片
image_folder = r"E:\碩士\財務工程(二)\圖片檔"

# 確保圖片資料夾存在
os.makedirs(image_folder, exist_ok=True)

# 讀取 Excel 檔案並創建圖表並插入圖片

def plot_and_insert_images(df, file_path, worksheet, start_row):
    # 選擇月增率和年增率欄位
    growth_col_1 = [col for col in df.columns if "月增 (%)" in col][0]  # 找到第一個「月增 (%)」
    growth_col_2 = [col for col in df.columns if "年增 (%)" in col][0]  # 找到第一個「年增 (%)」

    # 將選擇的欄位轉換為數字
    df[growth_col_1] = pd.to_numeric(df[growth_col_1], errors="coerce")
    df[growth_col_2] = pd.to_numeric(df[growth_col_2], errors="coerce")

    # 將「月別」欄位轉換為日期格式
    df['月別'] = pd.to_datetime(df['月別'], errors='coerce')

    # **圖表 1：單月月增 (%)**
    plt.figure(figsize=(10, 5))
    plt.plot(df['月別'], df[growth_col_1], marker="o", linestyle="-", color="b", label="單月月增 (%)")
    plt.title(f"單月月增 (%) 變化趨勢 - {file_path}")
    plt.xlabel("年份")
    plt.ylabel("月增 (%)")
    plt.gca().xaxis.set_major_locator(YearLocator())  # 每年顯示一次
    plt.gca().xaxis.set_major_formatter(DateFormatter('%Y'))  # 顯示年份（%Y）
    plt.xticks(rotation=45)
    plt.legend()
    plt.grid(True)

    # 儲存圖片到指定資料夾
    image_file = os.path.join(image_folder, f"{os.path.basename(file_path)}_growth_month.png")
    plt.savefig(image_file, format='png', dpi=100)
    plt.close()  # 關閉圖表，避免記憶體占用

    # 插入圖片到 Excel
    img = openpyxl.drawing.image.Image(image_file)
    img.width, img.height = 800, 400  # 設定圖片大小
    worksheet.add_image(img, f"A{start_row}")  # 插入圖片，動態設定單元格位置

    # **圖表 2：年增率 (%)**
    plt.figure(figsize=(10, 5))
    plt.plot(df['月別'], df[growth_col_2], marker="s", linestyle="-", color="g", label="年增率 (%)")
    plt.title(f"年增率 (%) 變化趨勢 - {file_path}")
    plt.xlabel("年份")
    plt.ylabel("年增率 (%)")
    plt.gca().xaxis.set_major_locator(YearLocator())  # 每年顯示一次
    plt.gca().xaxis.set_major_formatter(DateFormatter('%Y'))  # 顯示年份（%Y）
    plt.xticks(rotation=45)
    plt.legend()
    plt.grid(True)

    # 儲存圖片到指定資料夾
    image_file = os.path.join(image_folder, f"{os.path.basename(file_path)}_growth_year.png")
    plt.savefig(image_file, format='png', dpi=100)
    plt.close()  # 關閉圖表，避免記憶體占用

    # 插入圖片到 Excel
    img = openpyxl.drawing.image.Image(image_file)
    img.width, img.height = 800, 400  # 設定圖片大小
    worksheet.add_image(img, f"A{start_row + 15}")  # 插入圖片，動態設定單元格位置

    # **圖表 3：單月月增 (%) 與年增率 (%) 合併**
    plt.figure(figsize=(10, 5))
    plt.plot(df['月別'], df[growth_col_1], marker="o", linestyle="-", color="b", label="單月月增 (%)")
    plt.plot(df['月別'], df[growth_col_2], marker="s", linestyle="-", color="g", label="年增率 (%)")
    plt.title(f"單月月增 (%) 與年增率 (%) 合併變化趨勢 - {file_path}")
    plt.xlabel("年份")
    plt.ylabel("增率 (%)")
    plt.gca().xaxis.set_major_locator(YearLocator())  # 每年顯示一次
    plt.gca().xaxis.set_major_formatter(DateFormatter('%Y'))  # 顯示年份（%Y）
    plt.xticks(rotation=45)
    plt.legend()
    plt.grid(True)

    # 儲存圖片到指定資料夾
    image_file = os.path.join(image_folder, f"{os.path.basename(file_path)}_growth_combined.png")
    plt.savefig(image_file, format='png', dpi=100)
    plt.close()  # 關閉圖表，避免記憶體占用

    # 插入圖片到 Excel
    img = openpyxl.drawing.image.Image(image_file)
    img.width, img.height = 800, 400  # 設定圖片大小
    worksheet.add_image(img, f"A{start_row + 35}")  # 插入圖片，動態設定單元格位置

# 檔案處理與插入圖片
def save_to_excel_with_images(file_paths, file_path4):
    # 使用 openpyxl 加載現有的 Excel 文件
    wb = openpyxl.load_workbook(file_path4)
    worksheet = wb.create_sheet("推薦公司營收增長表")  # 新建一個名為 "圖表" 的工作表

    start_row = 1  # 設定第一張圖表的起始行數

    # 處理每個原始 Excel 檔案
    for file_path in file_paths:
        df = pd.read_excel(file_path, header=None, dtype=str)  # 讀取 Excel，不設定標題

        # 合併前 4 行作為標題，去除 nan
        df.columns = df.iloc[:4].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=0)
        df = df[4:].reset_index(drop=True)  # 刪除前 4 行

        # 去除重複欄位名稱
        df = df.loc[:, ~df.columns.duplicated()]

        # 插入圖表
        plot_and_insert_images(df, file_path, worksheet, start_row)

        # 更新下一個圖表的起始行數
        start_row += 40  # 每個圖表間隔 45 行

    # 保存修改後的 Excel 檔案
    wb.save(file_path4)
    print(f"圖片已成功插入並儲存到 {file_path4}！")

# 設定 Excel 檔案路徑
save_to_excel_with_images(file_paths, file_path4)


圖片已成功插入並儲存到 E:\碩士\財務工程(二)\ZF_score_PE5.xlsx！


# 推薦公司營收相比表

In [17]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter, YearLocator
import openpyxl
from io import BytesIO
import matplotlib
from matplotlib import font_manager

# 設定中文字型 (根據您系統的字型調整)
font_path = 'C:/Windows/Fonts/msyh.ttc'  # 微軟正黑體（根據您的系統路徑調整）
font_prop = font_manager.FontProperties(fname=font_path)


# 檔案路徑
file_paths = [
    r"E:\碩士\財務工程(二)\飛捷.xlsx",
    r"E:\碩士\財務工程(二)\映泰.xlsx"
]

# 設定新的 Excel 檔案路徑
file_path4 = r"E:\碩士\財務工程(二)\ZF_score_PE5.xlsx"

# 目標資料夾，用來儲存圖片
image_folder = r"E:\碩士\財務工程(二)\圖片檔"

# 確保圖片資料夾存在
os.makedirs(image_folder, exist_ok=True)

def plot_and_insert_images1(df1, df2, file_path1, file_path2, worksheet, start_row):
    # 選擇月增率和年增率欄位
    growth_col_1 = [col for col in df1.columns if "月增 (%)" in col][0]  # 找到第一個「月增 (%)」
    growth_col_2 = [col for col in df1.columns if "年增 (%)" in col][0]  # 找到第一個「年增 (%)」

    # 將選擇的欄位轉換為數字
    df1[growth_col_1] = pd.to_numeric(df1[growth_col_1], errors="coerce")
    df1[growth_col_2] = pd.to_numeric(df1[growth_col_2], errors="coerce")

    df2[growth_col_1] = pd.to_numeric(df2[growth_col_1], errors="coerce")
    df2[growth_col_2] = pd.to_numeric(df2[growth_col_2], errors="coerce")

    # 將「月別」欄位轉換為日期格式
    df1['月別'] = pd.to_datetime(df1['月別'], errors='coerce')
    df2['月別'] = pd.to_datetime(df2['月別'], errors='coerce')


    
    # **圖表 1：單月月增 (%)**
    plt.figure(figsize=(10, 5))
    plt.plot(df1['月別'], df1[growth_col_1], marker="o", linestyle="-", color="b", label="飛捷 單月月增 (%)")
    plt.plot(df2['月別'], df2[growth_col_1], marker="o", linestyle="-", color="r", label="映泰 單月月增 (%)")
    plt.title(f"單月月增 (%) 相比變化趨勢 - {file_path1} vs {file_path2}", fontsize=14, fontproperties=font_prop)  # 設定標題並使用指定字型
    plt.xlabel("年份", fontsize=12, fontproperties=font_prop)
    plt.ylabel("月增 (%)", fontsize=12, fontproperties=font_prop)
    plt.gca().xaxis.set_major_locator(YearLocator())  # 每年顯示一次
    plt.gca().xaxis.set_major_formatter(DateFormatter('%Y'))  # 顯示年份（%Y）
    plt.xticks(rotation=45)
    plt.legend()
    plt.grid(True)

        # 保存圖片到指定資料夾
    image_file = os.path.join(image_folder, f"{os.path.basename(file_path)}_growth_compare_month.png")
    plt.savefig(image_file, format='png', dpi=100)
    plt.close()  # 關閉圖表，避免記憶體占用

    # 保存圖片至 BytesIO
    img_stream = BytesIO()
    plt.savefig(img_stream, format='png', dpi=100)  # 降低 DPI 以適應 Excel
    img_stream.seek(0)
    plt.close()  # 關閉圖表，避免記憶體占用

    # 插入圖片到 Excel
    img = openpyxl.drawing.image.Image(img_stream)
    img.width, img.height = 800, 400  # 設定圖片大小
    worksheet.add_image(img, f"A{start_row}")  # 插入圖片，動態設定單元格位置

    # **圖表 2：年增率 (%)**
    plt.figure(figsize=(10, 5))
    plt.plot(df1['月別'], df1[growth_col_2], marker="s", linestyle="-", color="b", label="飛捷 年增率 (%)")
    plt.plot(df2['月別'], df2[growth_col_2], marker="s", linestyle="-", color="r", label="映泰 年增率 (%)")
    plt.title(f"年增率 (%) 相比趨勢 - {file_path1} vs {file_path2}", fontsize=14, fontproperties=font_prop)  # 設定標題並使用指定字型
    plt.xlabel("年份", fontsize=12, fontproperties=font_prop)
    plt.ylabel("年增率 (%)", fontsize=12, fontproperties=font_prop)
    plt.gca().xaxis.set_major_locator(YearLocator())  # 每年顯示一次
    plt.gca().xaxis.set_major_formatter(DateFormatter('%Y'))  # 顯示年份（%Y）
    plt.xticks(rotation=45)
    plt.legend()
    plt.grid(True)

            # 保存圖片到指定資料夾
    image_file = os.path.join(image_folder, f"{os.path.basename(file_path)}_growth_compare_year.png")
    plt.savefig(image_file, format='png', dpi=100)
    plt.close()  # 關閉圖表，避免記憶體占用
    
    # 保存圖片至 BytesIO
    img_stream = BytesIO()
    plt.savefig(img_stream, format='png', dpi=100)  # 降低 DPI 以適應 Excel
    img_stream.seek(0)
    plt.close()  # 關閉圖表，避免記憯體占用

    # 插入圖片到 Excel
    img = openpyxl.drawing.image.Image(img_stream)
    img.width, img.height = 800, 400  # 設定圖片大小
    worksheet.add_image(img, f"A{start_row + 15}")  # 插入圖片，動態設定單元格位置


# 檔案處理與插入圖片
def save_to_excel_with_images(file_path4):
    # 使用 openpyxl 加載現有的 Excel 文件
    wb = openpyxl.load_workbook(file_path4)
    worksheet = wb.create_sheet("推薦公司營收相比表")  # 新建一個名為 "圖表" 的工作表

    start_row = 1  # 設定第一張圖表的起始行數

    # 處理每個原始 Excel 檔案
    for i in range(0, len(file_paths), 2):  # 假設每次處理兩個公司（檔案）
        file_path1 = file_paths[i]
        file_path2 = file_paths[i+1]  # 假設是兩家公司比較

        df1 = pd.read_excel(file_path1, header=None, dtype=str)  # 讀取 Excel，不設定標題
        df2 = pd.read_excel(file_path2, header=None, dtype=str)  # 讀取第二個 Excel，不設定標題

        # 合併前 4 行作為標題，去除 nan
        df1.columns = df1.iloc[:4].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=0)
        df2.columns = df2.iloc[:4].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=0)
        df1 = df1[4:].reset_index(drop=True)  # 刪除前 4 行
        df2 = df2[4:].reset_index(drop=True)  # 刪除前 4 行

        # 去除重複欄位名稱
        df1 = df1.loc[:, ~df1.columns.duplicated()]
        df2 = df2.loc[:, ~df2.columns.duplicated()]

        # 插入圖表
        plot_and_insert_images1(df1, df2, file_path1, file_path2, worksheet, start_row)

        # 更新下一個圖表的起始行數
        start_row += 45  # 每個圖表間隔 45 行

    # 保存修改後的 Excel 檔案
    wb.save(file_path4)
    print(f"圖片已成功插入並儲存到 {file_path4}！")

# 設定 Excel 檔案路徑
save_to_excel_with_images(file_path4)

圖片已成功插入並儲存到 E:\碩士\財務工程(二)\ZF_score_PE5.xlsx！
